In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_excel('cleaned_labeled_dataset.xlsx')
df

,text,label,clean_text
0,Kunjungan Prabowo ini untuk meresmikan dan men...,Sumber Daya Alam,kunjungan prabowo meresmikan menyerahkan proye...
1,RT Anies dapat tepuk tangan meriah saat jadi R...,Politik,anies tepuk tangan meriah rektor mewajibkan ma...
2,@CIqXqwGAT04tMtx4OCATxjoVq7vv/Y8HeYaIOgMFg8Y= ...,Demografi,memang bener memang pendukung goblok pendukung...
3,RT @L3R8XFBw3WGbxRPSj0/0hHZTbqVGX7qtfwRg9zmhK7...,Politik,anies bersikap kritis kinerja prabowo dianggap...
4,Anies Baswedan Harap ASN termasuk TNI dan Polr...,Politik,anies baswedan harap asn tni polri pegang sump...
...,...,...,...
4279,RT @l1DsGGe2xObT3t72dHwqlT58X7jvEYtEnauZIZSYwS...,Politik,ngeliat debat kemaren prabowo kicep kekira kel...
4280,Masyarakat yakin bahwa Prabowo-Gibran memiliki...,Politik,masyarakat prabowo gibran memiliki visi sejala...
4281,imo both are irrational but yg satu jauh lebih...,Ekonomi,in my opinion both are irrational but irration...
4282,@cIIGSdjaPrmAQY1E4gWnLpIZSdyQn8ZMhjJzgOsxfRM= ...,Pertahanan dan Keamanan,look at that ganjar berkecimpung lgislatif eks...


In [3]:
null_index = df[df.isnull().any(axis=1)].index
df.loc[null_index]


,text,label,clean_text
72,@+0HCSSnYXfZ/WiNzDKgSO08o/BKbymxlncndey4/N+E= ...,Sosial Budaya,NaN


In [4]:
#drop index 72
df = df.drop(null_index)

In [16]:
X = df['clean_text']
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_train.shape, X_test.shape, y_train.shape, y_test.shape


((3426,), (857,), (3426,), (857,))

In [17]:
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

estimators = [
    # ('xgboost', XGBClassifier()),
    # ('adaboot', AdaBoostClassifier()),
    # ('catboost', CatBoostClassifier(verbose=0)),
    # ('lightgbm', LGBMClassifier()),
    ('randomforest', RandomForestClassifier()),
    ('logisticregression', LogisticRegression()),
    ('svm', SVC()),
    ('naivebayes', MultinomialNB())
]

stacking = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())
stacking.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


StackingClassifier(estimators=[('randomforest', RandomForestClassifier()),
                               ('logisticregression', LogisticRegression()),
                               ('svm', SVC()),
                               ('naivebayes', MultinomialNB())],
                   final_estimator=LogisticRegression())

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import balanced_accuracy_score

y_pred = stacking.predict(X_test)
print(classification_report(y_test, y_pred))
print(balanced_accuracy_score(y_test, y_pred))

                         precision    recall  f1-score   support

              Demografi       0.00      0.00      0.00        11
                Ekonomi       0.71      0.62      0.66        55
               Geografi       0.00      0.00      0.00         4
               Ideologi       1.00      0.24      0.38        51
Pertahanan dan Keamanan       0.65      0.37      0.47        54
                Politik       0.78      0.95      0.85       575
          Sosial Budaya       0.51      0.34      0.41        79
       Sumber Daya Alam       0.67      0.29      0.40        28

               accuracy                           0.75       857
              macro avg       0.54      0.35      0.40       857
           weighted avg       0.74      0.75      0.72       857

0.34989485384609864


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
